# A. Build a baseline model

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error as the loss function.

In [1]:
import numpy as np
import pandas as pd

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [5]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

n_cols = predictors.shape[1] # number of predictors

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [7]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

1\. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3)
predictors_train.shape

(721, 8)

2\. Train the model on the training data using 50 epochs.

In [10]:
model = regression_model()
model.fit(predictors_train, target_train, epochs=50)

Epoch 1/50
23/23 [==============================] - 0s 650us/step - loss: 68836.9141
Epoch 2/50
23/23 [==============================] - 0s 722us/step - loss: 17153.1406
Epoch 3/50
23/23 [==============================] - 0s 651us/step - loss: 3176.5022
Epoch 4/50
23/23 [==============================] - 0s 693us/step - loss: 1825.1464
Epoch 5/50
23/23 [==============================] - 0s 694us/step - loss: 1691.6249
Epoch 6/50
23/23 [==============================] - 0s 643us/step - loss: 1572.0203
Epoch 7/50
23/23 [==============================] - 0s 781us/step - loss: 1456.9655
Epoch 8/50
23/23 [==============================] - 0s 608us/step - loss: 1352.8481
Epoch 9/50
23/23 [==============================] - 0s 650us/step - loss: 1254.4751
Epoch 10/50
23/23 [==============================] - 0s 607us/step - loss: 1161.8158
Epoch 11/50
23/23 [==============================] - 0s 651us/step - loss: 1080.4862
Epoch 12/50
23/23 [==============================] - 0s 650us/step - los

3\. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

In [11]:
from sklearn.metrics import mean_squared_error

In [12]:
predictions_test = model.predict(predictors_test)
mean_squared_error(target_test, predictions_test)

391.2648504786464

4\. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [13]:
def evaluate_model(reg_model, predictors, train, epoch):
    mean_squared_errors = []
    for i in range(50):
        model = reg_model
        predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3)
        model.fit(predictors_train, target_train, epochs=epoch, verbose=0)
        
        predictions_test = model.predict(predictors_test)
        
        mse = mean_squared_error(target_test, predictions_test)
        mean_squared_errors.append(mse)
        
    return np.mean(mean_squared_errors), np.std(mean_squared_errors)

In [14]:
mean_mse, std_mse = evaluate_model(model, predictors, target, 50)
mean_mse, std_mse

(62.09029525252533, 33.379116657661264)

# B. Normalize the data (5 marks)

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [15]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [16]:
mean_mse, std_mse = evaluate_model(model, predictors_norm, target, 50)
mean_mse, std_mse

(54.43148623010025, 68.49536754384394)

How does the mean of the mean squared errors compare to that from Step A?
- Compared to A, mean of MSE is reduced.

# C. Increase the number of epochs (5 marks)

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

In [17]:
mean_mse, std_mse = evaluate_model(model, predictors_norm, target, 100)
mean_mse, std_mse

(29.16310457674105, 3.0825759346618375)

- Mean of MSE is smaller than B.

# D. Increase the number of hidden layers (5 marks)

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

In [18]:
def regression_model2():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model2 = regression_model2()

In [19]:
mean_mse, std_mse = evaluate_model(model2, predictors_norm, target, 50)
mean_mse, std_mse

(30.9314008188113, 17.33306706387002)

How does the mean of the mean squared errors compare to that from Step B?
- Compared to B, mean of MSE is smaller.